# n元语言模型回退算法

本次作业要求补全本笔记中的n元语言模型的采用Good-Turing折扣的Katz回退算法。

### 预处理

首先创建一些预处理函数。

引入必要的模块，定义些类型别名。

In [1]:
import re
import itertools

from typing import List, Dict, Tuple

Sentence = List[str]
IntSentence = List[int]

Corpus = List[Sentence]
IntCorpus = List[IntSentence]

Gram = Tuple[int]

下面的函数用于将文本正则化并词元化。该函数会将所有英文文本转为小写，去除文本中所有的标点，简单起见将所有连续的数字用一个`N`代替，将形如`let's`的词组拆分为`let`和`'s`两个词。

In [2]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")
def normaltokenize(corpus: List[str]) -> Corpus:
    """
    Normalizes and tokenizes the sentences in `corpus`. Turns the letters into
    lower case and removes all the non-alphadigit characters and splits the
    sentence into words and added BOS and EOS marks.

    Args:
        corpus - list of str

    Return:
        list of list of str where each inner list of str represents the word
          sequence in a sentence from the original sentence list
    """

    tokeneds = [ ["<s>"]
               + list(
                   filter(lambda tkn: len(tkn)>0,
                       _splitor_pattern.split(
                           _digit_pattern.sub("N", stc.lower()))))
               + ["</s>"]
                    for stc in corpus
               ]
    return tokeneds

接下来定义两个函数用来从训练语料中构建词表，并将句子中的单词从字符串表示转为整数索引表示。

In [3]:
def extract_vocabulary(corpus: Corpus) -> Dict[str, int]:
    """
    Extracts the vocabulary from `corpus` and returns it as a mapping from the
    word to index. The words will be sorted by the codepoint value.

    Args:
        corpus - list of list of str

    Return:
        dict like {str: int}
    """

    vocabulary = set(itertools.chain.from_iterable(corpus))
    vocabulary = dict(
            map(lambda itm: (itm[1], itm[0]),
                enumerate(
                    sorted(vocabulary))))
    return vocabulary

def words_to_indices(vocabulary: Dict[str, int], sentence: Sentence) -> IntSentence:
    """
    Convert sentence in words to sentence in word indices.

    Args:
        vocabulary - dict like {str: int}
        sentence - list of str

    Return:
        list of int
    """

    return list(map(lambda tkn: vocabulary.get(tkn, len(vocabulary)), sentence))

接下来读入训练数据，将数据预处理。

In [4]:
import functools

with open("data/news.2007.en.shuffled.deduped.train") as f:
    texts = list(map(lambda l: l.strip(), f.readlines()))

print("Loaded training set.")

corpus = normaltokenize(texts)
vocabulary = extract_vocabulary(corpus)
corpus = list(
        map(functools.partial(words_to_indices, vocabulary),
            corpus))

print("Preprocessed training set.")

Loaded training set.
Preprocessed training set.


### 设计模型

参照公式

$$
P_{\text{bo}}(w_k | W_{k-n+1}^{k-1}) = \begin{cases}
    d(W_{k-n+1}^k) \dfrac{C(W_{k-n+1}^k)}{C(W_{k-n+1}^{k-1})} &  C(W_{k-n+1}^k) > 0 \\
    \alpha(W_{k-n+1}^{k-1}) P_{\text{bo}}(w_k | W_{k-n+2}^{k-1}) &  \text{否则} \\
\end{cases}
$$

实现n元语言模型及采用Good-Turing折扣的Katz回退算法。

需要实现的功能包括：

1. 统计各词组（gram）在训练语料中的频数
2. 计算同频词组个数$N_r$
3. 计算$d(W_{k-n+1}^k)$
4. 计算$\alpha(W_{k-n+1}^{k-1})$
5. 根据公式计算回退概率
6. 计算概率对数与困惑度（PPL）

$d$与$\alpha$如何计算可以参考作业文件中的算法说明以及[SRILM](http://www.speech.sri.com/projects/srilm/)的[`ngram-discount(7)`手册页](http://www.speech.sri.com/projects/srilm/manpages/ngram-discount.7.html)。

In [5]:
import math

class NGramModel:
    def __init__(self, vocab_size: int, n: int = 4):
        """
        Constructs `n`-gram model with a `vocab_size`-size vocabulnextry.

        Args:
            vocab_size - int
            n - int
        """

        self.vocab_size: int = vocab_size
        self.n: int = n

        self.frequencies: List[Dict[Gram, int]] = [{} for _ in range(n)]
        self.disfrequencies: List[Dict[Gram, int]] = [{} for _ in range(n)]
        
        self.ncounts: Dict[ Gram, Dict[int, int]] = {}
        self.discount_threshold:int = 7
        # we only need to return a float instead of a tuple, so I modified the code here
        self._d: Dict[Gram, float] = {} 
        self._alpha: List[Dict[Gram, float]] = [{} for _ in range(n)]
        self.eps = 1e-10

    def learn(self, corpus: IntCorpus):
        """
        Learns the parameters of the n-gram model.

        Args:
            corpus - list of list of int
        """
        for stc in corpus:
            for i in range(1, len(stc)+1):
                for j in range(min(i, self.n)):
                    # TODO: count the frequencies of the grams
                    
                    gram = tuple(stc[i - j - 1: i])
                    if gram not in self.frequencies[j]:
                        self.frequencies[j][gram] = 1
                    else:
                        self.frequencies[j][gram] += 1
                        
        for i in range(1, self.n):
            grams = itertools.groupby(
                    sorted(
                        sorted(
                            map(lambda itm: (itm[0][:-1], itm[1]),
                                 self.frequencies[i].items()),
                               key=(lambda itm: itm[1])),
                        key=(lambda itm: itm[0])))
            # TODO: calculates the value of $N_r$
            
            for key, elem in grams:
                gram, num, l = key[0], key[1], len(list(elem))
                if gram not in self.ncounts:
                    self.ncounts[gram]={}
                self.ncounts[gram][num]=l

    def d(self, gram: Gram) -> float:
        """
        Calculates the interpolation coefficient.

        Args:
            gram - tuple of int

        Return:
            float
        """

        if gram not in self._d:
            # TODO: calculates the value of $d'$

            n = len(gram)-1
            theta, r = self.discount_threshold, self.frequencies[n][gram]
            gram1 = gram[:-1]
            
            if self.frequencies[n][gram] > theta or 1 not in self.ncounts[gram1]:
                self._d[gram]=1
            else:
                self.key_check(gram1, theta + 1)
                self.key_check(gram1, r + 1)  
                divisor = self.ncounts[gram1][1] - (theta + 1) * self.ncounts[gram1][theta + 1]
                lamda = self.ncounts[gram1][1]
                if divisor != 0:
                    lamda /= divisor
                self._d[gram] = lamda * (r + 1) * self.ncounts[gram1][r + 1]/(r * self.ncounts[gram1][r]) + 1 - lamda
        return self._d[gram]

    def key_check(self, gram, p):
        if p not in self.ncounts[gram]:
            self.ncounts[gram][p] = 0
        
    def alpha(self, gram: Gram) -> float:
        """
        Calculates the back-off weight alpha(`gram`)

        Args:
            gram - tuple of int

        Return:
            float
        """

        n = len(gram)
        if gram not in self._alpha[n]:
            if gram in self.frequencies[n-1]:
                # TODO: calculates the value of $\alpha$
                
                numerator = 1.
                denominator = 1.
                for key in self.frequencies[n]:
                    if key[:-1] == gram:
                        numerator -= self[key]
                        denominator -= self[key[1:]]
                self._alpha[n][gram] = numerator/denominator
                
            else:
                self._alpha[n][gram] = 1.
        return self._alpha[n][gram]

    def __getitem__(self, gram: Gram) -> float:
        """
        Calculates smoothed conditional probability P(`gram[-1]`|`gram[:-1]`).

        Args:
            gram - tuple of int

        Return:
            float
        """

        n = len(gram)-1
        if gram not in self.disfrequencies[n]:
            if n>0:
                # TODO: calculates the smoothed probability value according to the formulae
                
                gram1 = tuple(list(gram)[:-1])
                gram2 = tuple(list(gram)[1:])
                if gram in self.frequencies[n]:
                    self.disfrequencies[n][gram] = self.d(gram) * self.frequencies[n][gram] / self.frequencies[n-1][gram1]
                else:
                    alpha = self.alpha(gram1) 
                    self.disfrequencies[n][gram] = alpha * self[gram2]
                
            else: # uni-gram, n == 0
                self.disfrequencies[n][gram] = \
                self.frequencies[n].get(gram, self.eps)/float(len(self.frequencies[0]))
        return self.disfrequencies[n][gram]

    def log_prob(self, sentence: IntSentence) -> float:
        """
        Calculates the log probability of the given sentence. Assumes that the
        first token is always "<s>".

        Args:
            sentence: list of int

        Return:
            float
        """

        log_prob = 0.
        for i in range(2, len(sentence) + 1):
            # TODO: calculates the log probability
            
            if i < 4:
                result = max(self.eps, self[tuple(sentence[:i])])
            else:
                result = max(self.eps, self[tuple(sentence[i-4: i])])
            log_prob += math.log(result)
        return log_prob / (len(sentence) - 1)

    def ppl(self, sentence: IntSentence) -> float:
        """
        Calculates the PPL of the given sentence. Assumes that the first token
        is always "<s>".

        Args:
            sentence: list of int

        Return:
            float
        """

        # TODO: calculates the PPL
        return math.exp(- self.log_prob(sentence))

### 训练与测试

现在数据与模型均已齐备，可以训练并测试了。

训练模型：

In [6]:
import pickle as pkl

model = NGramModel(len(vocabulary))
model.learn(corpus)
with open("model.pkl", "wb") as f:
    pkl.dump(vocabulary, f)
    pkl.dump(model, f)

print("Dumped model.")

Dumped model.


在测试集上测试计算困惑度：

In [7]:
import pickle as pkl
with open("model.pkl", "rb") as f:
    vocabulary = pkl.load(f)
    model = pkl.load(f)
print("Loaded model.")

with open("data/news.2007.en.shuffled.deduped.test") as f:
    test_set = list(map(lambda l: l.strip(), f.readlines()))
test_corpus = normaltokenize(test_set)
test_corpus = list(
        map(functools.partial(words_to_indices, vocabulary),
            test_corpus))
ppls = []
for t in test_corpus:
    ppls.append(model.ppl(t))
    print(ppls[-1])
print("Avg: ", sum(ppls)/len(ppls))

Loaded model.
14797.748235365625
23359.215029721872
37869.412568324944
16386.678052874544
16179.770607906454
207.4609797942948
7321452.9596323315
650.013211111779
35037.850313012605
14541.395154791113
536.0787400583525
2309.8790684023224
936824.9651420182
119297.32757801784
69662.44580345663
151323.43977043396
7298.75580577709
240597.39620811096
37960.90274211718
1081.85893675652
30308.820515113708
69889.16207549049
33806.21585068515
276184.01321388106
7415.842693775885
80090.65828770152
44185.38336910625
398.1968170791759
14849.638719665929
5016.933159163419
49358.47063634666
180158.35988052536
26095.514293755772
4159.390516765648
132519.0423447606
13160.299120149124
119849.66591029812
51966.86634192761
232402.74249073234
326503.8046218399
30439.32177407148
153487.11212147694
7877.684033106953
320345.6731942347
51399.70403766408
1101328.6607328209
61155.59681846324
12010.676791304268
156553.68183916074
352781.23164290906
Avg:  259861.4783484873
